# import needed libraries 


In [1]:
from nltk.corpus import stopwords,reuters
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
stop_words = stopwords.words("english")
from sklearn.metrics import accuracy_score , classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import re
import string

# Represent train and test docs


In [2]:
#extracting train and test documents
#train_documents
train_docs_id = list(filter(lambda doc: doc.startswith("train"),reuters.fileids()));
train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
print('Number of docs in the train set: ' + str(len(train_docs)))
#test_documents
test_docs_id = list(filter(lambda doc: doc.startswith("test"),reuters.fileids()));
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]
print('Number of docs in the test set: ' + str(len(test_docs)))

Number of docs in the train set: 7769
Number of docs in the test set: 3019


# Cleaning Documents


In [3]:
codes = ['\r', '\n', '\t','lt']
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower() #Make text lowercase
    text = re.sub('\[.*?\]', '',str(text))
    text = re.sub('[%s]' % re.escape(string.punctuation), '', str(text)) #remove punctuation
    text = re.sub('\w*\d\w*', '', str(text)) #remove words containing numbers
    text = re.sub('[‘’“”…]', '', str(text))
    text = re.sub(r'dlrs', 'dollar', text)  # replace dlrs abreviation 
    text = re.sub(r'pct', 'percent', text)  # replace pct abreviation  
    for code in codes:
        text = re.sub(code, ' ', text)  # get rid of escape codes
    text = re.sub('\s+', ' ', text) # replace multiple spacess with one space   
    return text
    

In [4]:
#cleaning training and testing documents
cleaned_train_documents = []
for i in range(0,len(train_docs)):    
    cleaned_train_documents.append(clean_text(str(train_docs[i])))
cleaned_test_documents = []
for i in range(0,len(test_docs)):    
    cleaned_test_documents.append(clean_text(str(test_docs[i])))

In [5]:
#cleaning training and testing documents
cleaned_train_documents = []
for i in range(0,len(train_docs)):    
    cleaned_train_documents.append(clean_text(str(train_docs[i])))
cleaned_test_documents = []
for i in range(0,len(test_docs)):    
    cleaned_test_documents.append(clean_text(str(test_docs[i])))

# Tozenization

In [6]:
stemmer=PorterStemmer()
def toknize(text):
    stop_words = set(stopwords.words("english"))
    tokens = word_tokenize(text) #split document into individual words(tokens)
    tokens= [word for word in tokens if word not in stop_words] #filter out stop words if requested
    tokens = [word for word in tokens if word.isalpha()] #remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if len(word) > 2] #filter out tokens that are one or two characters long
    tokens = [word for word in tokens if len(word) < 21] # filter out tokens that are more than twenty characters long
    stemmas= [stemmer.stem(word) for word in tokens]
    # recreate the document string from parsed words
    text = ''
    for stem in stemmas:
        text = text + ' ' + stem
    return tokens,text

In [7]:
train_text = [] # list of document strings for sklearn TF-IDF
train_tokens = []  # list of token lists
for doc in cleaned_train_documents:
    text_string = doc
    # parse words one at a time in document string
    tokens,text_string = toknize(text_string)
    train_tokens.append(tokens)
    train_text.append(text_string)

In [8]:
test_tokens = []  # list of token lists
test_text = [] # list of document strings for sklearn TF-IDF
for doc in cleaned_test_documents:
    text_string = doc
    # parse words one at a time in document string
    tokens,text_string = toknize(text_string)
    test_tokens.append(text_string)
    test_text.append(text_string)

# Applying (TF-IDF vectorizer)and(document frequency)


In [12]:
# Learn and transform train documents
vectorizer = TfidfVectorizer(tokenizer=word_tokenize,max_df=0.95, min_df=3,max_features=1000)
vectorised_train_documents = vectorizer.fit_transform(train_text)
# transform test documents
vectorised_test_documents = vectorizer.transform(test_text)

# Transform Multilabel labels


In [9]:
#encode multi-label per instance 
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([reuters.categories(doc_id)
for doc_id in train_docs_id])
test_labels = mlb.transform([reuters.categories(doc_id)
for doc_id in test_docs_id])

# Train and Evaluate Classifiers


In [10]:
ModelsPerformance = {}

def metricsReport(modelName, test_labels, predictions):
    accuracy = accuracy_score(test_labels, predictions)

    macro_precision = precision_score(test_labels, predictions, average='macro')
    macro_recall = recall_score(test_labels, predictions, average='macro')
    macro_f1 = f1_score(test_labels, predictions, average='macro')

    micro_precision = precision_score(test_labels, predictions, average='micro')
    micro_recall = recall_score(test_labels, predictions, average='micro')
    micro_f1 = f1_score(test_labels, predictions, average='micro')
    hamLoss = hamming_loss(test_labels, predictions)
    print("------" + modelName + " Model Metrics-----")
    print("Accuracy: {:.4f}\nHamming Loss: {:.4f}\nPrecision:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nRecall:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nF1-measure:\n  - Macro: {:.4f}\n  - Micro: {:.4f}"\
          .format(accuracy, hamLoss, macro_precision, micro_precision, macro_recall, micro_recall, macro_f1, micro_f1))
    ModelsPerformance[modelName] = micro_f1

# Random Forest Classifier

In [13]:
rfClassifier = RandomForestClassifier(n_jobs=-1)
rfClassifier.fit(vectorised_train_documents, train_labels)
rfPreds = rfClassifier.predict(vectorised_test_documents)
metricsReport("Random Forest", test_labels, rfPreds)

C:\Users\HP\anaconda_new\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Random Forest Model Metrics-----
Accuracy: 0.7082
Hamming Loss: 0.0051
Precision:
  - Macro: 0.4024
  - Micro: 0.9576
Recall:
  - Macro: 0.1383
  - Micro: 0.6568
F1-measure:
  - Macro: 0.1855
  - Micro: 0.7792


# Decision Tree Classifier


In [14]:
dtClassifier = DecisionTreeClassifier()
dtClassifier.fit(vectorised_train_documents, train_labels)
dtPreds = dtClassifier.predict(vectorised_test_documents)
metricsReport("Decision Tree", test_labels, dtPreds)

C:\Users\HP\anaconda_new\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------Decision Tree Model Metrics-----
Accuracy: 0.7271
Hamming Loss: 0.0066
Precision:
  - Macro: 0.3557
  - Micro: 0.7752
Recall:
  - Macro: 0.3119
  - Micro: 0.7369
F1-measure:
  - Macro: 0.3226
  - Micro: 0.7556


# Support Vector Machine Classifier 

In [15]:
svmClassifier = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
svmClassifier.fit(vectorised_train_documents, train_labels)

svmPreds = svmClassifier.predict(vectorised_test_documents)
metricsReport("SVC Sq. Hinge Loss", test_labels, svmPreds)

C:\Users\HP\anaconda_new\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------SVC Sq. Hinge Loss Model Metrics-----
Accuracy: 0.8003
Hamming Loss: 0.0036
Precision:
  - Macro: 0.5803
  - Micro: 0.9407
Recall:
  - Macro: 0.3353
  - Micro: 0.7917
F1-measure:
  - Macro: 0.3974
  - Micro: 0.8598


In [16]:
powerSetSVC = LabelPowerset(LinearSVC())
powerSetSVC.fit(vectorised_train_documents, train_labels)

powerSetSVCPreds = powerSetSVC.predict(vectorised_test_documents)
metricsReport("Power Set SVC", test_labels, powerSetSVCPreds)

------Power Set SVC Model Metrics-----
Accuracy: 0.8370
Hamming Loss: 0.0039
Precision:
  - Macro: 0.5990
  - Micro: 0.8929
Recall:
  - Macro: 0.4080
  - Micro: 0.8130
F1-measure:
  - Macro: 0.4608
  - Micro: 0.8511


C:\Users\HP\anaconda_new\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Base Classifier

In [17]:
nbClassifier = OneVsRestClassifier(MultinomialNB())
nbClassifier.fit(vectorised_train_documents, train_labels)

nbPreds = nbClassifier.predict(vectorised_test_documents)
metricsReport("Multinomial NB", test_labels, nbPreds)

------Multinomial NB Model Metrics-----
Accuracy: 0.6870
Hamming Loss: 0.0055
Precision:
  - Macro: 0.2388
  - Micro: 0.8973
Recall:
  - Macro: 0.1260
  - Micro: 0.6765
F1-measure:
  - Macro: 0.1446
  - Micro: 0.7714


C:\Users\HP\anaconda_new\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
print("  Model Name " + " "*10 + "| Micro-F1 Score")
print("-------------------------------------------")
for key, value in ModelsPerformance.items():
    print("  " + key, " "*(20-len(key)) + "|", value)
    print("-------------------------------------------")

  Model Name           | Micro-F1 Score
-------------------------------------------
  Random Forest        | 0.7791508238276298
-------------------------------------------
  Decision Tree        | 0.7555798986717789
-------------------------------------------
  SVC Sq. Hinge Loss   | 0.8597534445250181
-------------------------------------------
  Power Set SVC        | 0.8511114217810708
-------------------------------------------
  Multinomial NB       | 0.771432922186691
-------------------------------------------
